In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/fast_nas_adapt/fast_nas_adapt/


In [ ]:
1

In [ ]:
!git clone -b bahleg-experiments https://github.com/bahleg/fast_nas_adapt.git

In [ ]:
#!rm -r fast_nas_adapt

In [ ]:
#!git checkout bahleg-experiments

In [ ]:
!pip install torchmetrics

In [ ]:
!rm *pckl

In [ ]:
import sys
sys.path.append('./src/')
import importlib
from matplotlib import pylab as plt

import numpy as np
import torch
import torchvision

import cifar_data
import utils #у меня есть utils библиотека, поэтому переименовала
import dartslike
import VGG19

import pickle
import os
import gc
import argparse
import dartslike

import importlib
import module2graph2
importlib.reload(module2graph2)
from torch.fx import symbolic_trace


In [ ]:
@torch.no_grad()
def safe_prune(model, trainloader, num_to_prune, gammas_to_prune):
    for x,y in trainloader:
        break
    num = 0
    for g in gammas_to_prune:
            model.gammas[g] = 1.0
    
    i = 0
    while num < num_to_prune and i < len(gammas_to_prune):
        model.gammas[i] = 0.0
  
        if abs(model(x) - model(torch.zeros(x.shape))).sum() < 1e-5:
            model.gammas[i] = 1.0
            i += 1
        else:
            num += 1
            i += 1
    if num == num_to_prune:
        return True
    else:
        return False 


In [ ]:

class LowRankLinear(torch.nn.Module):
    def __init__(self, in_, out_, dim=1):
        super().__init__()
        self.l = torch.nn.Parameter(torch.randn(in_, dim)*1e-3)
        self.r = torch.nn.Parameter(torch.randn(dim, out_) * 1e-3)
    
    def forward(self, x):
        #print (x.shape, self.l.shape, self.r.shape)
        return x@self.l@self.r


class Aux(torch.nn.Module):
    def __init__(self, sizes, layer_names):
        super().__init__()
        self.layers = torch.nn.ModuleList()
        self.layer_names = layer_names
        self.means_int = torch.nn.ModuleDict()
        self.lsigmas_int = torch.nn.ParameterDict()
        self.means_y = torch.nn.ModuleDict()
        self.lsigmas_y = torch.nn.ParameterDict()
        
        for i in range(len(layer_names)-1):
            current = layer_names[i]
            next_ = layer_names[i+1]
            mat_size = np.prod(sizes[current][1:]) * np.prod(sizes[next_][1:])
            if mat_size > 1024 * 1024:
                linear = LowRankLinear(np.prod(sizes[current][1:]), np.prod(sizes[next_][1:]))
            else:
                linear = torch.nn.Linear(np.prod(sizes[current][1:]), np.prod(sizes[next_][1:]))
            
            lsigma = torch.tensor(-2.0)
            self.means_int.update({current: linear})
            self.lsigmas_int.update({current: lsigma})
            self.layers.append(linear)
            
        for i in range(len(layer_names)):
            current = layer_names[i]
            linear = torch.nn.Linear(np.prod(sizes[current][1:]), 2)
            lsigma = torch.nn.Parameter(torch.tensor(-2.0))
            self.means_y.update({current: linear})
            self.lsigmas_y.update({current: lsigma})
            self.layers.append(linear)
            

In [ ]:
#!rm *pckl

In [ ]:

for seed in [0, 13, 42]:
    for parameter_opt_type in ['MI', 'CE']:
        for gamma_opt_type in ['MI', 'CE']:
            for lam in [0.0, 0.25, 0.5, 0.75, 1.0][::-1]:
                if gamma_opt_type == 'CE' and parameter_opt_type == 'CE' and lam != 0:
                    continue
                name = f'{parameter_opt_type}.{gamma_opt_type}.{lam}.{seed}'
                if os.path.exists(name+'.pckl'):
                  continue
                batch_size = 64
                device = 'cuda:0'
                trial_num = 1
                epoch_num = 50
                lr = 1e-3
                wd = 1e-6
                batch_seen = 5
                trainloader, valloader, testloader = cifar_data.get_dataloaders([0,1], batch_size=batch_size, need_val=True, cifar100 = True)
                model = VGG19.VGG19(num_classes = 10).to(device)
                model.load_state_dict(torch.load('../VGG19_CIFAR10.ckpt', map_location=device))
                model.model.classifier[6] = torch.nn.Linear(4096, 2)
                traced_graph = symbolic_trace(model)
                module2graph2.make_gamma_hooks(model, traced_graph, lambda: module2graph2.SigmoidGamma(0.0))
                model.to(device)
                sizes = {}
                model(torch.randn(64, 3, 33, 33).to(device))
                for k,v in model.intermediate.items():
                    sizes[k] = v.shape
                layer_names = [k for k in sizes]

                
                print (name)
                torch.manual_seed(seed)
                np.random.seed(seed)
                def intermediate_getter(x):
                    out = model(x)[0]
                    inter = model.intermediate
                    return out, inter 
                aux = Aux(sizes, layer_names).to(device)

                dl = dartslike.DartsLikeTrainer(model, parameter_optimization=parameter_opt_type, gamma_optimization=gamma_opt_type,
                                                    aux=aux,MI_Y_lambda=lam, layer_wise=False)
            
                history = dl.train_loop(trainloader, valloader, testloader, batch_seen, epoch_num, lr, lr, device, wd, model,
                                        intermediate_getter = intermediate_getter)

                with open(name+'.pckl','wb') as out:
                    #out.write(pickle.dumps( (history, model.cpu().state_dict())))
                    out.write(pickle.dumps( (history)))



In [ ]:
import pickle
import matplotlib.pylab as plt
import numpy as np 
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 9)
plt.xlabel('gamma_optimization')
plt.ylabel('accuracy')

for parameter_opt_type in ['MI', 'CE']:
    for gamma_opt_type in ['MI', 'CE']:
        for lam in [0.0, 0.25, 0.5, 0.75, 1.0][::-1]:
          results = []
          name = f'{parameter_opt_type}.{gamma_opt_type}.{lam}.{seed}'
          if gamma_opt_type == 'CE' and parameter_opt_type == 'CE' and lam != 0:
                continue

          for seed in [0, 13, 42]:
            
            
            with open(name+'.pckl', 'rb') as inp:
              results.append(pickle.loads(inp.read()))
          plt.plot(np.mean(results, 0))
          plt.fill_between(list(range(len(results[0]))), np.mean(results, 0) - np.std(results, 0), np.mean(results, 0) + np.std(results, 0), label= f'{parameter_opt_type}.{gamma_opt_type}.{lam}', alpha=0.5)
plt.legend(loc='best')
            


In [ ]:
np.std(results, 0)

In [ ]:
results

In [ ]:
result = model(torch.randn(64, 3, 33, 33).to(device))

In [ ]:
!git pull

In [ ]:
for g in model.named_modules():
  if 'gamma' in g[0].lower():
    print (g[0], list(g[1].parameters()))

In [ ]:
len(model.gammas)

In [ ]:
for g in model.gammas:
  print (g)